In [1]:
import mars.dataframe as md # https://pypi.org/project/pymars/
import pandas as pd
from datetime import datetime
import time
from multiprocessing import Pool
import os

In [56]:
data_path = "F:/Projects/Python/data_process/csv/tg_zd_machine-04.csv"
data_all = md.read_csv(data_path).to_pandas()
data_all.index = pd.DatetimeIndex(data_all["UPDATE_DATE"])
data_all = data_all.sort_index()

In [57]:
data_all.head()

,MACHINE_ID,ONTIME,WORKTIME,STOPTIME,QTY,PASS_QTY,REJECTS_QTY,BEAT,EFFICIENCY,STATUS,UPDATE_DATE,PDLINE_ID
UPDATE_DATE,,,,,,,,,,,,
2020-04-01 07:17:41,10,23,0,23,0,NaN,0.0,409,0.0,0,2020/4/1 7:17:41,NaN
2020-04-01 07:18:05,10,47,0,47,0,NaN,0.0,409,0.0,0,2020/4/1 7:18:05,NaN
2020-04-01 07:19:13,11,88,0,88,0,NaN,0.0,33,0.0,0,2020/4/1 7:19:13,NaN
2020-04-01 07:19:17,3,146,0,0,0,NaN,0.0,139,0.0,0,2020/4/1 7:19:17,NaN
2020-04-01 07:19:30,12,0,0,0,0,NaN,0.0,3809,0.0,0,2020/4/1 7:19:30,NaN


In [58]:
# 总数据大小
data_all.shape[0]

5537184

In [59]:
# 拆分装配线以及检测线
data_assembly = data_all.loc[data_all.PDLINE_ID.notnull()]
data_checking = data_all.loc[data_all.PDLINE_ID.isnull()]
print("length of assembly line data:{0}".format(data_assembly.shape[0]))
print("length of checking line data:{0}".format(data_checking.shape[0]))

length of assembly line data:3069551
length of checking line data:2467633


In [60]:
# 拆分数据输出.csv
def split_csv(data_csv):
    if(2 in data_csv["PDLINE_ID"].values.tolist()):
        for i in range(1,13):
            output = data_csv[data_csv["MACHINE_ID"].isin([i])]
            output.to_csv('F:/Projects/Python/data_process/csv/splited_data/data_assembly_{0}.csv'.format(i),
                          index=None)
    else:
        for i in range(1,13):
            output = data_csv[data_csv["MACHINE_ID"].isin([i])]
            output.to_csv('F:/Projects/Python/data_process/csv/splited_data/data_checking_{0}.csv'.format(i),
                          index=None)          

In [62]:
# 输出.csv别手贱运行！
# split_csv(data_assembly)
# split_csv(data_checking)

In [63]:
# 批量输入拆分后的数据
file_dir = 'F:/Projects/Python/data_process/csv/splited_data'
file_list = []
data_dict = {}
for root, dirs, files in os.walk(file_dir):  
    for file in files:  
            if os.path.splitext(file)[1] == '.csv':  
                file_list.append(file)

for file in file_list:
    data_in = md.read_csv('F:/Projects/Python/data_process/csv/splited_data/{0}'.format(file)).to_pandas()
    data_in.index = pd.DatetimeIndex(data_in["UPDATE_DATE"])
    print("The length of {0}: {1}.".format(file,data_in.shape[0]))
    data_dict["{0}".format(file)] = data_in

The length of data_assembly_1.csv: 4389.
The length of data_assembly_10.csv: 303791.
The length of data_assembly_11.csv: 303961.
The length of data_assembly_12.csv: 313935.
The length of data_assembly_2.csv: 304524.
The length of data_assembly_3.csv: 307289.
The length of data_assembly_4.csv: 304287.
The length of data_assembly_5.csv: 4656.
The length of data_assembly_6.csv: 308159.
The length of data_assembly_7.csv: 303956.
The length of data_assembly_8.csv: 304361.
The length of data_assembly_9.csv: 306243.
The length of data_checking_1.csv: 322915.
The length of data_checking_10.csv: 263207.
The length of data_checking_11.csv: 344964.
The length of data_checking_12.csv: 322976.
The length of data_checking_2.csv: 349780.
The length of data_checking_3.csv: 268791.
The length of data_checking_4.csv: 1365.
The length of data_checking_5.csv: 292293.
The length of data_checking_6.csv: 0.
The length of data_checking_7.csv: 269611.
The length of data_checking_8.csv: 18670.
The length of dat

In [10]:
# 丢弃工作时间错误的数据
def process_error_data_01(data):   
    for index, row in data.iterrows():
        if row['ONTIME'] != row['WORKTIME'] + row['STOPTIME']:
#             print(index, row['MACHINE_ID'] ,row['PDLINE_ID'])
            data.drop(index = index, axis=0, inplace=True)

In [11]:
# 单核处理
start = time.time()
process_error_data_01(data_tianzheng_assembly)
end = time.time()
print('Processing time %0.2f seconds.' % (end - start))

Processing time 214.02 seconds.


In [3]:
# 得到文件中的日期
# https://www.cnblogs.com/nxf-rabbit75/p/10662025.html
def get_date_list(data):
    process_data = data.copy()
    process_data = process_data.resample('D').mean()
    process_data = process_data.dropna(axis=0,how="all")
    process_date_list = process_data.index.strftime('%Y-%m-%d').tolist() # https://www.runoob.com/python/att-time-strftime.html

    #  用完删除，释放内存
    del process_data
    import gc
    gc.collect()
    
    return process_date_list

In [142]:
# 判断后一天开始时的QTY是否清零，即是否小于前一天结束时的QTY
def process_error_data_02(data, date_list):
    error = False
    for i in range(len(date_list)-1):
        QTY_01 = int(data.loc[date_list[i]][-1:]['QTY'])
        QTY_02 = int(data.loc[date_list[i+1]][:1]['QTY'])        
        if QTY_02 >= QTY_01:
#             判断开始结束的时间先后，思路错误，不需要判断
#             d1 = data.loc[date_list[i]][-1:].index.strftime("%H/%M/%S").values[0]
#             d2 = data.loc[date_list[i+1]][:1].index.strftime("%H/%M/%S").values[0]
#             d1 = datetime.strptime(d1, "%H/%M/%S")
#             d2 = datetime.strptime(d2, "%H/%M/%S")
#             if d2 > d1:
            print("* QTY of the day before:\n",data.loc[date_list[i]][-1:][['QTY']],'\n')
            print("* QTY of the day after:\n",data.loc[date_list[i+1]][:1][['QTY']],'\n')
            error = True
    if error == False:
        print("QTY data are all correct!\n")

In [143]:
for key in data_dict:
    print("**********"+ key + "**********")
    process_error_data_02(data_dict[key], get_date_list(data_dict[key]))

**********data_assembly_1.csv**********
QTY data are all correct!

**********data_assembly_10.csv**********
* QTY of the day before:
                       QTY
UPDATE_DATE              
2020-04-01 12:45:24  3910 

* QTY of the day after:
                       QTY
UPDATE_DATE              
2020-04-16 13:21:00  5227 

**********data_assembly_11.csv**********
* QTY of the day before:
                       QTY
UPDATE_DATE              
2020-04-01 12:45:26  3903 

* QTY of the day after:
                       QTY
UPDATE_DATE              
2020-04-16 13:21:00  5217 

**********data_assembly_12.csv**********
* QTY of the day before:
                       QTY
UPDATE_DATE              
2020-04-01 13:45:16  4551 

* QTY of the day after:
                       QTY
UPDATE_DATE              
2020-04-16 13:21:00  5205 

**********data_assembly_2.csv**********
* QTY of the day before:
                       QTY
UPDATE_DATE              
2020-04-01 12:52:18  4396 

* QTY of the day after:
       